<a href="https://colab.research.google.com/github/1NetoDev/HabituAI/blob/main/HabituAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Célula 1: Instalações e Configuração da API Key

# Instalações de bibliotecas Python
!pip install -q gradio google-generativeai markdown2 Pillow PyPDF2 pytesseract
# Adicionado markdown2 aqui ^ (Pillow, PyPDF2, Pytesseract são opcionais se não usar upload de arquivos)

# ... (resto da Célula 1 como antes, com a configuração da API Key) ...
print("Instalações de libs Python concluídas.") # Certifique-se que markdown2 foi listado

# Configuração da API Key do Google AI
import os
from google.colab import userdata
import google.generativeai as genai

GOOGLE_API_KEY_GLOBAL = None
try:
    GOOGLE_API_KEY_GLOBAL = userdata.get('GOOGLE_API_KEY')
    if GOOGLE_API_KEY_GLOBAL is None:
        raise ValueError("Chave 'GOOGLE_API_KEY' não encontrada nos Secrets do Colab.")
    genai.configure(api_key=GOOGLE_API_KEY_GLOBAL)
    print("Chave da API do Google AI configurada com sucesso para HabituAI!")
except Exception as e:
    print(f"ATENÇÃO: Erro ao configurar a chave da API do Google AI: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
Instalações de libs Python concluídas.
Chave da API do Google AI configurada com sucesso para HabituAI!


In [2]:
# Célula 2: Definições de Backend do HabituAI (Versão Limpa)

import google.generativeai as genai
import PyPDF2 # Mantido caso você reintroduza leitura de PDF em algum fluxo futuro, senão pode remover
from PIL import Image # Mantido caso reintroduza leitura de Imagem
import pytesseract # Mantido caso reintroduza leitura de Imagem
# import json # Removido pois não está em uso ativo

# --- Variáveis Globais de Estado da Sessão ---
# (Estas serão gerenciadas principalmente pela UI Gradio via gr.State,
# mas são definidas aqui para clareza da lógica do backend)
conversa_com_ia_para_habitos = []
plano_de_habitos_gerado = []
xp_total = 0
gold_total = 0
nivel_usuario = 1
id_habito_counter = 0

# --- Instruções de Sistema para a IA ---
INSTRUCAO_SISTEMA_BOAS_VINDAS = """
Você é 'HabituAI', seu Guia Pessoal para a Jornada de Hábitos.
Sua primeira tarefa é dar as boas-vindas ao 'Aventureiro' (usuário) de forma amigável, temática (RPG/jornada) e concisa.
Faça UMA pergunta clara e direta para que ele comece a compartilhar o que deseja trabalhar.
Exemplo de pergunta: "Saudações, nobre Aventureiro! Sou HabituAI, e juntos vamos forjar hábitos lendários. Para começar nossa jornada, diga-me: qual grande meta você busca ou qual desafio de hábito quer conquistar primeiro?"
Use Markdown para formatação (ex: **negrito**). Mantenha a resposta inicial curta (2-3 frases mais a pergunta).
"""

INSTRUCAO_SISTEMA_ANALISE_E_PLANOS = """
Você é 'HabituAI', Guia Pessoal para a Jornada de Hábitos. O Aventureiro compartilhou informações ou está pedindo um novo plano.
Sua tarefa agora é:
1.  Analisar TODO o histórico da conversa para entender o contexto atual e os objetivos do Aventureiro.
2.  Se o Aventureiro estiver pedindo um novo conjunto de missões ou se as informações anteriores forem suficientes para um *novo* plano (ou um plano atualizado), SUGIRA de 2 a 4 'Missões' (hábitos).
    *   Se já existia um plano, você pode sugerir missões completamente novas OU variações/progressões das anteriores, conforme o contexto da conversa.
    *   Para cada Missão/Hábito:
        - Dê um nome temático (Ex: '**Missão: Despertar do Fênix**').
        - Defina uma META CLARA e ALCANÇAVEL (Ex: 'Acordar 15 minutos mais cedo para alongamento').
        - Atribua XP (entre 10 e 50) e GOLD (entre 1 e 5).
        - Especifique a FREQUÊNCIA (Ex: 'Diário', 'Seg, Qua, Sex').
3.  Se as informações ainda forem muito vagas para criar missões concretas, faça MAIS UMA ou DUAS perguntas MUITO ESPECÍFICAS para clarificar.
4.  SEMPRE que você apresentar um conjunto de missões (sejam iniciais ou um novo conjunto), APRESENTE este plano com o marcador EXATO "### 📜 Seu Novo Pergaminho de Missões 📜 ###" no início da seção do plano.
    - Cada missão deve começar com `**Missão: [Nome da Missão]**`.
    - Os detalhes devem ser itens de lista: `- Meta: [descrição]`, `- Frequência: [descrição]`, `- Recompensa: [XP] XP, [GOLD] Gold`.
5.  Após apresentar o plano (ou fazer mais perguntas), finalize sua resposta PERGUNTANDO ao Aventureiro se ele está pronto para estas novas missões, se deseja ajustes, ou respondendo à sua última pergunta.
Use Markdown para toda a formatação.
"""

# --- Função de Interação com a IA ---
def chamar_ia_para_habitos(historico_conversa_atual_lista, nova_mensagem_usuario_str, instrucao_sistema_especifica_str, nome_modelo_str="gemini-1.5-flash"):
    # A API Key já deve estar configurada via genai.configure() na Célula 1
    try:
        model = genai.GenerativeModel(
            model_name=nome_modelo_str,
            system_instruction=instrucao_sistema_especifica_str
        )

        prompt_construido_para_api = []
        for item in historico_conversa_atual_lista:
            prompt_construido_para_api.append(f"{item['role'].title()}: {item['content']}")
        prompt_construido_para_api.append(f"User: {nova_mensagem_usuario_str}")
        prompt_completo_str = "\n\n".join(prompt_construido_para_api)

        response = model.generate_content(prompt_completo_str)

        if not hasattr(response, 'text') or not response.text:
             novo_historico_com_erro = historico_conversa_atual_lista + [{"role": "user", "content": nova_mensagem_usuario_str}, {"role": "model", "content": "ErroAPI_RespostaInesperadaOuVazia"}]
             return "ErroAPI_RespostaInesperadaOuVazia", novo_historico_com_erro

        resposta_ia_texto_str = response.text
        novo_historico_lista = historico_conversa_atual_lista + [{"role": "user", "content": nova_mensagem_usuario_str}, {"role": "model", "content": resposta_ia_texto_str}]
        return resposta_ia_texto_str, novo_historico_lista

    except AttributeError as ae:
        msg_erro_api = "ErroAPI_ChaveNaoConfiguradaOuInvalida"
        if "'NoneType' object has no attribute 'send_request'" in str(ae) or \
           "configure() has not been called" in str(ae) or \
           "'GoogleGenerativeAI' object has no attribute 'default_transport'" in str(ae):
            print(f"DEBUG Backend: Erro crítico - API Key provavelmente não configurada: {ae}") # Manter este debug é útil
        else:
            msg_erro_api = f"ErroAPI_Atributo: {str(ae)}"
        novo_historico_com_erro_attr = historico_conversa_atual_lista + [{"role": "user", "content": nova_mensagem_usuario_str}, {"role": "model", "content": msg_erro_api}]
        return msg_erro_api, novo_historico_com_erro_attr
    except Exception as e:
        error_details = getattr(e, 'message', str(e))
        if hasattr(e, 'response') and hasattr(e.response, 'text'):
            error_details += f" | Detalhes da API: {e.response.text}"
        msg_erro_api_geral = f"ErroAPI_Comunicacao: {error_details}"
        novo_historico_com_erro_geral = historico_conversa_atual_lista + [{"role": "user", "content": nova_mensagem_usuario_str}, {"role": "model", "content": msg_erro_api_geral}]
        return msg_erro_api_geral, novo_historico_com_erro_geral

# --- Funções de Parseamento e Gamificação ---
def tentar_parsear_plano_de_habitos_da_ia(texto_resposta_ia_str):
    global plano_de_habitos_gerado, id_habito_counter

    marcador_plano = "### 📜 Seu Novo Pergaminho de Missões 📜 ###"
    # Para retrocompatibilidade com o prompt de exemplo da IA que você tinha antes
    marcador_plano_antigo = "### PERGAMINHO DAS MISSÕES INICIAIS ###"

    texto_relevante_para_parsear = ""
    if marcador_plano in texto_resposta_ia_str:
        texto_relevante_para_parsear = texto_resposta_ia_str.split(marcador_plano, 1)[-1]
    elif marcador_plano_antigo in texto_resposta_ia_str:
        texto_relevante_para_parsear = texto_resposta_ia_str.split(marcador_plano_antigo, 1)[-1]
    else:
        return False

    plano_de_habitos_gerado = [] # Limpa o plano anterior
    id_habito_counter = 0      # Reseta o contador de ID para o novo plano
    habitos_extraidos_temp = []

    linhas = texto_relevante_para_parsear.split('\n')
    missao_atual_nome_temp = None
    detalhes_habito_atual_temp = {}

    for linha_str in linhas:
        linha_strip = linha_str.strip()
        if not linha_strip: continue

        if linha_strip.startswith("**Missão:"):
            if missao_atual_nome_temp and 'meta' in detalhes_habito_atual_temp:
                id_habito_counter += 1
                detalhes_habito_atual_temp['id'] = id_habito_counter
                detalhes_habito_atual_temp['concluido'] = False
                detalhes_habito_atual_temp.setdefault('frequencia', 'Não especificada')
                detalhes_habito_atual_temp.setdefault('xp', 20)
                detalhes_habito_atual_temp.setdefault('gold', 2)
                habitos_extraidos_temp.append(detalhes_habito_atual_temp)
            missao_atual_nome_temp = linha_strip.replace("**Missão:", "").replace("**", "").strip()
            detalhes_habito_atual_temp = {"nome": missao_atual_nome_temp}
        elif missao_atual_nome_temp and linha_strip.startswith("- Meta:"):
            detalhes_habito_atual_temp['meta'] = linha_strip.replace("- Meta:", "").strip()
        elif missao_atual_nome_temp and linha_strip.startswith("- Frequência:"):
            detalhes_habito_atual_temp['frequencia'] = linha_strip.replace("- Frequência:", "").strip()
        elif missao_atual_nome_temp and linha_strip.startswith("- Recompensa:"):
            recompensa_str = linha_strip.replace("- Recompensa:", "").strip().lower()
            try:
                xp_val, gold_val = 20, 2
                partes_recompensa = recompensa_str.split(',')
                for parte in partes_recompensa:
                    parte_strip = parte.strip()
                    if "xp" in parte_strip: xp_val = int("".join(filter(str.isdigit, parte_strip)))
                    elif "gold" in parte_strip: gold_val = int("".join(filter(str.isdigit, parte_strip)))
                detalhes_habito_atual_temp['xp'] = xp_val
                detalhes_habito_atual_temp['gold'] = gold_val
            except ValueError:
                detalhes_habito_atual_temp['xp'], detalhes_habito_atual_temp['gold'] = 20, 2
        elif missao_atual_nome_temp and not linha_strip.startswith(("-", "**")):
            if 'meta' in detalhes_habito_atual_temp:
                 id_habito_counter += 1
                 detalhes_habito_atual_temp['id'] = id_habito_counter
                 detalhes_habito_atual_temp['concluido'] = False
                 detalhes_habito_atual_temp.setdefault('frequencia', 'Não especificada'); detalhes_habito_atual_temp.setdefault('xp', 20); detalhes_habito_atual_temp.setdefault('gold', 2)
                 habitos_extraidos_temp.append(detalhes_habito_atual_temp)
            missao_atual_nome_temp = None
            detalhes_habito_atual_temp = {}
            break

    if missao_atual_nome_temp and 'meta' in detalhes_habito_atual_temp:
        id_habito_counter += 1
        detalhes_habito_atual_temp['id'] = id_habito_counter
        detalhes_habito_atual_temp['concluido'] = False
        detalhes_habito_atual_temp.setdefault('frequencia', 'Não especificada'); detalhes_habito_atual_temp.setdefault('xp', 20); detalhes_habito_atual_temp.setdefault('gold', 2)
        habitos_extraidos_temp.append(detalhes_habito_atual_temp)

    if habitos_extraidos_temp:
        plano_de_habitos_gerado = habitos_extraidos_temp
        return True
    return False

def get_estado_jogo_atualizado():
    global xp_total, gold_total, nivel_usuario, plano_de_habitos_gerado
    nomes_habitos = [h.get("nome", f"Missão ID {h.get('id')}") for h in plano_de_habitos_gerado]
    habitos_concluidos_nomes = [h.get("nome") for h in plano_de_habitos_gerado if h.get("concluido", False)]
    return {
        "xp": xp_total, "gold": gold_total, "nivel": nivel_usuario,
        "habitos_detalhados": plano_de_habitos_gerado,
        "nomes_habitos_para_checkboxgroup": nomes_habitos,
        "habitos_concluidos_para_checkboxgroup": habitos_concluidos_nomes
    }

def marcar_habitos_concluidos_ui(lista_nomes_habitos_concluidos_da_ui):
    global xp_total, gold_total, plano_de_habitos_gerado, nivel_usuario

    for habito_dict in plano_de_habitos_gerado:
        nome_habito_no_plano = habito_dict.get("nome")
        estava_concluido = habito_dict.get("concluido", False)
        esta_na_lista_ui = nome_habito_no_plano in lista_nomes_habitos_concluidos_da_ui

        xp_habito = habito_dict.get("xp", 0)
        gold_habito = habito_dict.get("gold", 0)

        if esta_na_lista_ui and not estava_concluido:
            habito_dict["concluido"] = True
            xp_total += xp_habito
            gold_total += gold_habito
        elif not esta_na_lista_ui and estava_concluido:
            habito_dict["concluido"] = False
            xp_total -= xp_habito
            gold_total -= gold_habito
            if xp_total < 0: xp_total = 0
            if gold_total < 0: gold_total = 0

    xp_necessario_base = 100 # XP base para o primeiro nível
    xp_para_proximo_nivel = nivel_usuario * xp_necessario_base # Aumenta a necessidade com o nível

    if xp_total >= xp_para_proximo_nivel and xp_para_proximo_nivel > 0 :
        # Loop para permitir múltiplos level ups se XP suficiente for ganho de uma vez
        while xp_total >= xp_para_proximo_nivel:
            nivel_usuario += 1
            xp_total -= xp_para_proximo_nivel # Deduz o XP usado para subir de nível
            xp_para_proximo_nivel = nivel_usuario * xp_necessario_base # Recalcula para o novo nível
            print(f"DEBUG Backend: LEVEL UP! Você alcançou o Nível {nivel_usuario}! XP atual: {xp_total}. Próximo nível em {xp_para_proximo_nivel} XP.")
        if xp_total < 0 : xp_total = 0 # Garante que o XP não fique negativo após dedução

    return get_estado_jogo_atualizado()

def resetar_jogo_e_conversa():
    global conversa_com_ia_para_habitos, plano_de_habitos_gerado, xp_total, gold_total, nivel_usuario, id_habito_counter
    conversa_com_ia_para_habitos = []
    plano_de_habitos_gerado = []
    xp_total = 0
    gold_total = 0
    nivel_usuario = 1
    id_habito_counter = 0
    print("DEBUG Backend: Jogo e conversa resetados.") # Manter este debug é útil
    return get_estado_jogo_atualizado()

print("Célula 2: Funções de Backend do HabituAI (Versão Limpa) DEFINIDAS.")

Célula 2: Funções de Backend do HabituAI (Versão Limpa) DEFINIDAS.


In [3]:
# Célula 2.1: Estado da Sessão e Constantes do Sistema (Com Novo Prompt Unificado)

# --- Variáveis Globais de Estado da Sessão ---
conversa_com_ia_para_habitos = []
plano_de_habitos_gerado = []
xp_total = 0
gold_total = 0
nivel_usuario = 1
id_habito_counter = 0
objetivo_principal_usuario = "" # Nova variável para armazenar o objetivo principal

# --- Instrução de Sistema Principal para o "Guardião das Missões" ---
INSTRUCAO_SISTEMA_GUARDAO_DAS_MISSOES = """
Você é o **Guardião das Missões**, um ser místico e experiente no universo de habitus e desenvolvimento pessoal, agindo como Mestre de RPG para o Aventureiro (o usuário). Sua missão é transformar os grandes objetivos do Aventureiro em jornadas manejáveis, forjando 'Missões' (hábitos concretos) que o guiarão em sua Grande Busca (objetivo principal). Você entende a psicologia por trás da construção de hábitos, da gamificação e da importância de ver o impacto de cada pequeno passo na jornada maior.

Suas responsabilidades e fluxo de interação são:

1.  **Interação Inicial (Se o histórico da conversa estiver vazio ou for a primeira mensagem do usuário):**
    *   Apresente-se brevemente como o Guardião das Missões.
    *   Faça UMA pergunta clara e aberta para que o Aventureiro compartilhe seu **Objetivo Principal** ou "Grande Busca".
    *   Exemplo: "Saudações, Aventureiro! Eu sou o Guardião das Missões, aqui para forjar sua jornada rumo à grandeza. Diga-me, qual é a sua **Grande Busca**? Que nobre objetivo você almeja alcançar?"
    *   Mantenha esta primeira interação concisa e focada em obter o Objetivo Principal. Não sugira missões ainda.

2.  **Análise Profunda do Contexto (Para todas as mensagens subsequentes):**
    *   SEMPRE comece analisando TODO o histórico da nossa conversa para compreender o contexto atual, os sucessos, os desafios passados e, crucialmente, o **Objetivo Principal** que o Aventureiro busca alcançar (Ex: Concluir o projeto X, aprender Habilidade Y, melhorar Saúde Z). Se o Objetivo Principal ainda não estiver claro, sua prioridade é clarificá-lo usando o passo 4.

3.  **Forjar a Jornada: Quebrando o Objetivo Principal (Se o Objetivo Principal estiver claro e for complexo):**
    *   Se o Objetivo Principal for muito grande ou vago, sua primeira tarefa após a clarificação é **SUGERIR um caminho**, quebrando-o em **etapas menores ou marcos intermediários** (Ex: Objetivo 'Escrever um Livro' -> Etapa 1: Esboçar Capítulos, Etapa 2: Escrever Rascunho do Capítulo 1...). Explique brevemente por que essa quebra torna a busca mais factível e menos intimidante.
    *   Com base no Objetivo Principal (ou na próxima Etapa da jornada), você irá **forjar 'Missões' (hábitos)** que, ao serem completadas, movem o Aventureiro em direção a esse objetivo/etapa atual. Pense na "caminhada de trás pra frente" - que pequenos hábitos diários/semanais levam ao próximo marco?

4.  **Clarificando a Jornada (Se Necessário, ANTES de gerar missões):**
    *   Se o Objetivo Principal ou a última interação do Aventureiro for muito vaga, obscura ou insuficiente para forjar missões concretas (ou para quebrar o objetivo em etapas), faça no MÁXIMO **DUAS perguntas MUITO DIRECIONADAS e claras** para obter as informações que faltam.
    *   Mantenha SEMPRE o tom temático e encorajador. (Ex: "Guardião precisa de mais clareza para iluminar seu caminho, Aventureiro. Sua Grande Busca por 'ser mais saudável' se assemelha mais a dominar a arte da nutrição élfica ou à disciplina dos treinos dos guerreiros anões? Detalhes nos ajudarão a forjar missões precisas!")
    *   Somente após obter clareza suficiente, prossiga para gerar/atualizar o Pergaminho de Missões.

5.  **Gerar e Gerenciar o Pergaminho de Missões:**
    *   Quando for apropriado gerar ou atualizar um plano de missões (seja o primeiro, um pedido de renovação, ou se o contexto indicar a necessidade de novas tarefas com base no Objetivo Principal/Etapa atual), SUGIRA de **2 a 6 Missões** de hábito concretas.
    *   Se já existia um plano (identificável pelo histórico), você pode:
        *   **Renovar/Repetir:** Sugerir as mesmas missões se o Aventureiro precisar consolidá-las.
        *   **Progredir:** Aumentar a dificuldade ou duração de missões existentes.
        *   **Adicionar:** Incluir novas missões que complementem as existentes ou abordem novas facetas do Objetivo Principal/Etapa.
        *   **Substituir:** Trocar missões antigas por novas.
    *   **Para CADA Missão/Hábito sugerido, você DEVE definir:**
        *   **Nome Temático Engajador:** (Ex: '**Missão: Hidratação Dracônica**', '**Missão: Forja da Escrita (Capítulo 1)**').
        *   **Meta Clara e Acionável:** (Ex: 'Beber 2 litros de água pura ao longo do dia', 'Escrever 500 palavras focadas do Capítulo 1', 'Dedicar 30 minutos ao estudo de algoritmos de IA').
        *   **Frequência:** (Ex: 'Diário, especialmente ao amanhecer', '3 vezes por semana - Seg, Qua, Sex, no período da tarde', 'Finais de semana, por pelo menos 1 hora').
        *   **Recompensa:** Atribua **XP (Pontos de Experiência)** entre 10 e 100 (varie conforme a dificuldade e impacto) e **GOLD (Moedas)** entre 1 e 10.
        *   **Conexão com a Grande Busca:** **ESSENCIAL.** Explique brevemente (1-2 frases) COMO completar esta pequena missão contribui DIRETAMENTE para o Objetivo Principal ou a Etapa atual da jornada do Aventureiro. (Ex: "Completar a 'Hidratação Dracônica' diariamente não só mantém sua vitalidade de aventureiro, mas também aguça sua mente para os desafios da sua Grande Busca por Conhecimento.")

6.  **Formato de Apresentação do Pergaminho:**
    *   SEMPRE que apresentar um conjunto de missões (novas ou atualizadas), inicie a seção com o marcador EXATO:
        `### 📜 Seu Novo Pergaminho de Missões 📜 ###`
    *   Abaixo deste marcador, liste cada missão usando Markdown para clareza:
        ```markdown
        **Missão: [Nome Temático da Missão]**
        - Meta: [Descrição clara e acionável da meta]
        - Frequência: [Frequência sugerida para o hábito]
        - Recompensa: [Número] XP, [Número] Gold
        - Conexão com a Grande Busca: [Explicação de como contribui para o Objetivo Principal/Etapa]
        ```
    *   Use negrito para títulos/nomes importantes e listas (`- `) para os detalhes de cada missão.

7.  **Finalização e Próximo Passo:**
    *   Após apresentar o pergaminho de missões (ou fazer perguntas de clarificação), finalize SEMPRE sua resposta com uma chamada para ação clara, convidando o Aventureiro a:
        *   Confirmar se aceita as missões apresentadas.
        *   Solicitar ajustes no pergaminho.
        *   Responder às perguntas de clarificação (se for o caso).
    *   Exemplo de finalização: "Este é o pergaminho que forjei para sua jornada atual, Aventureiro. Ele ressoa com sua bravura? Diga-me se aceita estas missões, ou se o Guardião deve realizar algum ajuste para melhor servir sua Grande Busca!"
    *   Mantenha o tom temático e encorajador até o fim.

**Diretrizes Adicionais para o Guardião das Missões:**
*   Mantenha a consistência do tema de aventura/RPG em toda a interação.
*   Seja adaptável: ajuste a dificuldade das missões e as recompensas com base no histórico do Aventureiro e no feedback dele.
*   Use Markdown de forma consistente para tornar as missões fáceis de ler e visualizar.
*   Se o usuário apenas agradecer ou der uma resposta curta que não indica claramente o próximo passo, gentilmente o guie de volta para a aceitação das missões ou para fornecer mais informações se você fez uma pergunta.
"""

print("Célula 2.1: Estado da Sessão e Constantes do Sistema (Guardião das Missões) DEFINIDOS.")

Célula 2.1: Estado da Sessão e Constantes do Sistema (Guardião das Missões) DEFINIDOS.


In [5]:
# Célula 2.2: Funções de Gamificação e Parseamento (Versão Limpa e Otimizada)

# As variáveis globais (xp_total, gold_total, etc.) são definidas na Célula 2.1
# e acessadas aqui usando a palavra-chave 'global'.

def get_estado_jogo_atualizado():
    """Retorna um dicionário com o estado atual do jogo para a UI."""
    global xp_total, gold_total, nivel_usuario, plano_de_habitos_gerado

    # Prepara a lista de nomes para o CheckboxGroup e quais estão marcados
    nomes_habitos_para_ui = [h.get("nome", f"Missão ID {h.get('id', 'Desconhecido')}") for h in plano_de_habitos_gerado]
    habitos_concluidos_para_ui = [h.get("nome") for h in plano_de_habitos_gerado if h.get("concluido", False)]

    return {
        "xp": xp_total,
        "gold": gold_total,
        "nivel": nivel_usuario,
        "habitos_detalhados": plano_de_habitos_gerado, # Lista completa de missões com todos os detalhes
        "nomes_habitos_para_checkboxgroup": nomes_habitos_para_ui,
        "habitos_concluidos_para_checkboxgroup": habitos_concluidos_para_ui
    }

def resetar_jogo_e_conversa():
    """Reseta todas as variáveis de estado do jogo e da conversa para o início."""
    global conversa_com_ia_para_habitos, plano_de_habitos_gerado, xp_total, gold_total, nivel_usuario, id_habito_counter, objetivo_principal_usuario

    conversa_com_ia_para_habitos = []
    plano_de_habitos_gerado = []
    xp_total = 0
    gold_total = 0
    nivel_usuario = 1
    id_habito_counter = 0
    objetivo_principal_usuario = "" # Reseta o objetivo principal também

    print("DEBUG Backend: Jogo, conversa e objetivo principal resetados.")
    return get_estado_jogo_atualizado() # Retorna o estado limpo

def tentar_parsear_plano_de_habitos_da_ia(texto_resposta_ia_str):
    """
    Tenta extrair as missões do texto da IA usando o marcador específico.
    Atualiza a variável global 'plano_de_habitos_gerado'.
    """
    global plano_de_habitos_gerado, id_habito_counter

    marcador_plano = "### 📜 Seu Novo Pergaminho de Missões 📜 ###"

    if marcador_plano not in texto_resposta_ia_str:
        # print(f"DEBUG Backend - tentar_parsear: Marcador '{marcador_plano}' não encontrado na resposta.")
        return False # Indica que não há um novo plano para parsear

    # Limpa o plano anterior e reseta o contador de ID antes de adicionar novos itens
    plano_de_habitos_gerado = []
    id_habito_counter = 0
    habitos_extraidos_temp = []

    # Pega apenas a parte do texto que vem APÓS o marcador do pergaminho
    try:
        parte_das_missoes = texto_resposta_ia_str.split(marcador_plano, 1)[1]
    except IndexError:
        # print(f"DEBUG Backend - tentar_parsear: Marcador encontrado, mas sem conteúdo após ele.")
        return False

    linhas = parte_das_missoes.split('\n')
    missao_atual_nome_temp = None
    detalhes_habito_atual_temp = {}

    for linha_str in linhas:
        linha_strip = linha_str.strip()
        if not linha_strip: # Pula linhas completamente vazias
            continue

        if linha_strip.startswith("**Missão:"):
            # Se uma missão anterior estava sendo montada e tinha uma meta, salva-a
            if missao_atual_nome_temp and 'meta' in detalhes_habito_atual_temp:
                id_habito_counter += 1
                detalhes_habito_atual_temp['id'] = id_habito_counter
                detalhes_habito_atual_temp['concluido'] = False # Novas missões começam não concluídas
                # Define defaults se não encontrados durante o parseamento
                detalhes_habito_atual_temp.setdefault('frequencia', 'Não especificada')
                detalhes_habito_atual_temp.setdefault('xp', 20)
                detalhes_habito_atual_temp.setdefault('gold', 2)
                detalhes_habito_atual_temp.setdefault('conexao_busca', 'A ser revelada pelo Guardião!') # Default para o novo campo
                habitos_extraidos_temp.append(detalhes_habito_atual_temp)

            # Começa uma nova missão
            missao_atual_nome_temp = linha_strip.replace("**Missão:", "").replace("**", "").strip()
            detalhes_habito_atual_temp = {"nome": missao_atual_nome_temp}

        elif missao_atual_nome_temp and linha_strip.startswith("- Meta:"):
            detalhes_habito_atual_temp['meta'] = linha_strip.replace("- Meta:", "").strip()
        elif missao_atual_nome_temp and linha_strip.startswith("- Frequência:"):
            detalhes_habito_atual_temp['frequencia'] = linha_strip.replace("- Frequência:", "").strip()
        elif missao_atual_nome_temp and linha_strip.startswith("- Recompensa:"):
            recompensa_str = linha_strip.replace("- Recompensa:", "").strip().lower()
            try:
                xp_val, gold_val = 20, 2 # Defaults
                partes_recompensa = recompensa_str.split(',')
                for parte in partes_recompensa:
                    parte_strip = parte.strip()
                    if "xp" in parte_strip: xp_val = int("".join(filter(str.isdigit, parte_strip)))
                    elif "gold" in parte_strip: gold_val = int("".join(filter(str.isdigit, parte_strip)))
                detalhes_habito_atual_temp['xp'] = xp_val
                detalhes_habito_atual_temp['gold'] = gold_val
            except (ValueError, IndexError): # Captura erro se o split falhar ou conversão para int
                # print(f"DEBUG Backend - tentar_parsear: Erro ao parsear recompensa '{recompensa_str}' para '{missao_atual_nome_temp}'. Usando defaults.")
                detalhes_habito_atual_temp['xp'], detalhes_habito_atual_temp['gold'] = 20, 2
        elif missao_atual_nome_temp and linha_strip.startswith("- Conexão com a Grande Busca:"): # NOVO CAMPO
            detalhes_habito_atual_temp['conexao_busca'] = linha_strip.replace("- Conexão com a Grande Busca:", "").strip()

        # Se encontrar uma linha que não é parte da estrutura esperada da missão,
        # e uma missão estava sendo parseada, assume-se que a lista de missões no formato esperado terminou.
        elif missao_atual_nome_temp and not (linha_strip.startswith("-") or linha_strip.startswith("```")):
            # print(f"DEBUG Backend - tentar_parsear: Linha '{linha_strip}' não corresponde ao formato de detalhe de missão. Encerrando parse de missões.")
            break

    # Adiciona a última missão que estava sendo parseada, se estiver completa
    if missao_atual_nome_temp and 'meta' in detalhes_habito_atual_temp:
        id_habito_counter += 1
        detalhes_habito_atual_temp['id'] = id_habito_counter
        detalhes_habito_atual_temp['concluido'] = False
        detalhes_habito_atual_temp.setdefault('frequencia', 'Não especificada')
        detalhes_habito_atual_temp.setdefault('xp', 20)
        detalhes_habito_atual_temp.setdefault('gold', 2)
        detalhes_habito_atual_temp.setdefault('conexao_busca', 'A ser revelada pelo Guardião!')
        habitos_extraidos_temp.append(detalhes_habito_atual_temp)

    if habitos_extraidos_temp:
        plano_de_habitos_gerado = habitos_extraidos_temp # Atualiza a variável global
        print(f"DEBUG Backend: NOVO Plano de hábitos parseado com {len(plano_de_habitos_gerado)} missão(ões).")
        return True # Sucesso
    else:
        # print("DEBUG Backend: Nenhum hábito concreto foi parseado da resposta da IA (após o marcador de pergaminho).")
        return False # Falha

def marcar_habitos_concluidos_ui(lista_nomes_habitos_concluidos_da_ui):
    """
    Atualiza o estado de conclusão dos hábitos e recalcula XP, Gold e Nível.
    """
    global xp_total, gold_total, plano_de_habitos_gerado, nivel_usuario

    # print(f"DEBUG Backend: marcar_habitos_concluidos_ui - Recebido da UI: {lista_nomes_habitos_concluidos_da_ui}")

    for habito_dict in plano_de_habitos_gerado:
        nome_habito_no_plano = habito_dict.get("nome")
        estava_concluido = habito_dict.get("concluido", False)
        # A lista_nomes_habitos_concluidos_da_ui contém os nomes dos hábitos que estão MARCADOS na UI
        esta_marcado_na_ui = nome_habito_no_plano in lista_nomes_habitos_concluidos_da_ui

        xp_habito = habito_dict.get("xp", 0)
        gold_habito = habito_dict.get("gold", 0)

        if esta_marcado_na_ui and not estava_concluido: # Hábito foi recém-marcado como concluído
            habito_dict["concluido"] = True
            xp_total += xp_habito
            gold_total += gold_habito
            # print(f"DEBUG Backend: Hábito '{nome_habito_no_plano}' MARCADO. +{xp_habito}XP, +{gold_habito}Gold")
        elif not esta_marcado_na_ui and estava_concluido: # Hábito foi recém-desmarcado
            habito_dict["concluido"] = False
            xp_total -= xp_habito
            gold_total -= gold_habito
            if xp_total < 0: xp_total = 0 # XP não pode ser negativo
            if gold_total < 0: gold_total = 0 # Gold não pode ser negativo
            # print(f"DEBUG Backend: Hábito '{nome_habito_no_plano}' DESMARCADO. Pontos revertidos.")

    # Lógica de Level Up
    xp_necessario_base = 100 # XP base para o primeiro nível
    xp_para_proximo_nivel = nivel_usuario * xp_necessario_base

    if xp_total >= xp_para_proximo_nivel and xp_para_proximo_nivel > 0:
        while xp_total >= xp_para_proximo_nivel: # Permite múltiplos level-ups
            xp_total -= xp_para_proximo_nivel # Deduz o XP usado para o nível atual
            nivel_usuario += 1
            xp_para_proximo_nivel = nivel_usuario * xp_necessario_base # Recalcula para o novo nível
            print(f"DEBUG Backend: LEVEL UP! Você alcançou o Nível {nivel_usuario}! XP restante: {xp_total}. Próximo nível em {xp_para_proximo_nivel} XP.")
    if xp_total < 0 : xp_total = 0 # Segurança final para XP não ser negativo

    return get_estado_jogo_atualizado()

print("Célula 2.2: Funções de Gamificação e Parseamento (Versão Limpa) DEFINIDAS.")

Célula 2.2: Funções de Gamificação e Parseamento (Versão Limpa) DEFINIDAS.


In [8]:
# Célula 3: Definição e Lançamento da Interface Gradio (Corrigida)

import gradio as gr
import html
import markdown2 # Para converter Markdown da IA para HTML no chat

# --- Definição do Tema ---
custom_theme_habituai = gr.themes.Base(
    primary_hue=gr.themes.colors.green,
    secondary_hue=gr.themes.colors.lime,
    neutral_hue=gr.themes.colors.gray,
    font=[gr.themes.GoogleFont("Inter"), "ui-sans-serif", "system-ui", "sans-serif"],
).set(
    body_background_fill="#1A1A1A", body_background_fill_dark="#1A1A1A",
    block_background_fill="#242424", block_border_width="0px", block_shadow="*shadow_drop_lg",
    button_primary_background_fill=gr.themes.colors.green.c600, button_primary_text_color="white",
)

# --- CSS ---
css_login_e_chat = """
#output_conversa_html_habituai { background-color: #202020; padding: 15px 10px; border-radius: 8px; overflow-y: auto; height: 450px; display: flex; flex-direction: column-reverse; }
#output_conversa_html_habituai::before { content: ''; flex-grow: 1; }
.message-wrapper { display: flex; flex-direction: column; margin-bottom: 12px; max-width: 78%; min-width: 60px; }
.sender-name { font-size: 0.75em; color: #A0A0A0; margin-bottom: 3px; }
.message-bubble { padding: 9px 13px; border-radius: 18px; line-height: 1.45; word-wrap: break-word; font-size: 0.92em; display: inline-block; }
.user-wrapper { align-self: flex-end; margin-left: auto; }
.user-wrapper .sender-name { text-align: right; padding-right: 13px; }
.message-bubble.user-bubble { background-color: #FFFFFF !important; color: #1D1D1D !important; border-radius: 18px 5px 18px 18px; align-self: flex-end; }
.model-wrapper { align-self: flex-start; margin-right: auto; }
.model-wrapper .sender-name { text-align: left; padding-left: 13px; }
.message-bubble.model-bubble { background-color: #25D366 !important; color: #FFFFFF !important; border-radius: 5px 18px 18px 18px; align-self: flex-start; }
.message-bubble.model-bubble strong { font-weight: bold; } .message-bubble.model-bubble em { font-style: italic; }
.message-bubble.model-bubble ul, .message-bubble.model-bubble ol { margin-left: 20px; padding-left: 5px; list-style-position: outside; margin-top: 0.5em; margin-bottom: 0.5em;}
.message-bubble.model-bubble li { margin-bottom: 4px; } .message-bubble.model-bubble p { margin-top: 0; margin-bottom: 0.5em; }
#chat_input_bar_habituai { background-color: #242424; padding: 8px; border-top: 1px solid #303030;}
#input_usuario_habituai textarea { border-radius: 18px !important; background-color: #2C2C2C !important; color: #E0E0E0 !important; border: 1px solid #383838 !important;}
#btn_enviar_habituai { border-radius: 50% !important; min-width: 45px !important; background-color: var(--button-primary-background-fill) !important; color: var(--button-primary-text-color) !important; border: none !important; font-size: 1.5em !important; width: 45px !important; height: 45px !important; display:flex; align-items:center; justify-content:center;}
.status-section { padding: 10px; background-color: var(--block-background-fill); border-radius: 8px; margin-bottom:15px;}
.status-section .gr-markdown h3 { margin-top:0; color: var(--primary-hue-500) !important;}
.gr-checkboxgroup label span {color: #E0E0E0 !important} .gr-checkboxgroup .gr-check-radio span {color: #C0C0C0 !important}
#detalhes_habitos_display_md ul { padding-left: 0; } /* Ajuste para remover padding extra da lista ul */
#detalhes_habitos_display_md ul li { background-color: #2C2C2C !important; border-radius: 4px !important; margin-bottom: 5px !important; padding: 8px !important; list-style-type: none; }
#detalhes_habitos_display_md ul li strong { color: #E0E0E0 !important; } #detalhes_habitos_display_md ul li small { color: #AAAAAA !important; }
#login_screen_wrapper { display: flex; flex-direction: column; justify-content: center; align-items: center; min-height: 50vh; padding-top: 20px; }
#login_block { padding: 25px 30px; max-width: 380px; width: 90%; background-color: var(--block-background-fill); border-radius: 10px; box-shadow: 0 4px 15px rgba(0,0,0,0.2); }
#login_block .gr-button-primary { width: 100%; margin-top:20px !important; padding: 10px !important; font-size: 1.05em !important;}
#login_error_message { color: #FF6B6B; margin-top: 15px; text-align: center; font-size:0.9em; min-height: 20px;}
#login_title { text-align:center; color: var(--primary-hue-500); margin-bottom:25px !important; font-size:1.8em; font-weight:bold; }
#app_header_habituai { padding: 10px 20px; background-color: #202020; color: #E0E0E0; display: flex; justify-content: space-between; align-items: center; }
#user_info_display { font-size: 0.9em; text-align: right; }
"""

# --- CREDENCIAIS SIMULADAS ---
USUARIO_VALIDO = "aventureiro"
SENHA_VALIDA = "habito123"

with gr.Blocks(theme=custom_theme_habituai, title="HabitMaster IGNITE", css=css_login_e_chat) as demo_habituai_com_login:

    usuario_logado_estado_ui = gr.State(False)
    nome_usuario_logado_estado_ui = gr.State("")
    estado_conversa_hist_ui = gr.State([])
    estado_jogo_completo_ui = gr.State({
        "xp": 0, "gold": 0, "nivel": 1, "habitos_detalhados": [],
        "nomes_habitos_para_checkboxgroup": [], "habitos_concluidos_para_checkboxgroup": []
    })

    with gr.Column(visible=True, elem_id="login_screen_wrapper") as tela_login_ui_wrapper:
        with gr.Group(elem_id="login_block"):
            gr.Markdown("⚔️ HabitMaster IGNITE ⚔️", elem_id="login_title")
            login_usuario_input_ui = gr.Textbox(label="Nome do Aventureiro", placeholder=f"Ex: {USUARIO_VALIDO}")
            login_senha_input_ui = gr.Textbox(label="Senha Secreta", placeholder=f"Ex: {SENHA_VALIDA}", type="password")
            btn_entrar_ui = gr.Button("Entrar na Guilda", variant="primary")
            login_status_msg_ui = gr.Markdown(elem_id="login_error_message")

    with gr.Column(visible=False) as tela_principal_habituai_ui:
        with gr.Row(elem_id="app_header_habituai"):
            gr.Markdown("<h1 style='margin:0; font-size: 1.6em; color: var(--primary-hue-500);'>⚔️ HabitMaster IGNITE ⚔️</h1>")
            user_info_display_ui = gr.Markdown(elem_id="user_info_display")

        with gr.Tabs():
            with gr.TabItem("💬 Chat com HabituAI / 🔥 Missões"):
                with gr.Row():
                    with gr.Column(scale=7):
                        output_conversa_html_ui = gr.HTML(elem_id="output_conversa_html_habituai", value="<p style='color:#A0A0A0; text-align:center; margin-top: 20px;'><em>HabituAI aguarda seu comando...</em></p>")
                        with gr.Row(elem_id="chat_input_bar_habituai", equal_height=False):
                            input_usuario_chat_ui = gr.Textbox(elem_id="input_usuario_habituai", placeholder="Sua resposta ao HabituAI...", lines=1, show_label=False, scale=10)
                            btn_enviar_mensagem_ui = gr.Button("➤", elem_id="btn_enviar_habituai", scale=1, min_width=45)
                    with gr.Column(scale=3):
                        with gr.Group(elem_classes="status-section"):
                            gr.Markdown("### 📜 Seu Pergaminho")
                            nivel_display_ui = gr.Markdown(f"**Nível:** 1")
                            xp_display_ui = gr.Markdown(f"**XP:** 0")
                            gold_display_ui = gr.Markdown(f"**Gold:** 0")
                        with gr.Group(elem_classes="status-section"):
                            gr.Markdown("### 🔥 Missões Ativas")
                            checkbox_habitos_ui_display = gr.CheckboxGroup(label="Marque as missões concluídas:", choices=[], value=[], interactive=True)
                            detalhes_habitos_display_markdown_ui = gr.Markdown("Nenhuma missão ativa no momento.", elem_id="detalhes_habitos_display_md")
            with gr.TabItem("🏰 Loja do Aventureiro"):
                 gr.Markdown("## 🛍️ Empório do Mestre dos Hábitos 🛍️\n<p style='text-align:center; font-size:1.5em; margin-top: 20px;'>🚧 EM CONSTRUÇÃO 🚧</p>")
        gr.Markdown("<hr><p style='text-align:center; font-size:0.9em; color:#757575;'>Uma criação da Imersão IA Alura com HabitMaster IGNITE</p>")

    def formatar_conversa_para_html(historico_lista_dict):
        chat_html = ""
        for item_dict in historico_lista_dict:
            is_user = item_dict.get("role") == "user"
            wrapper_class = "user-wrapper" if is_user else "model-wrapper"; bubble_class = "user-bubble" if is_user else "model-bubble"
            sender_name = "Você" if is_user else "HabituAI"; raw_content = item_dict.get('content', '')
            processed_content = html.escape(raw_content).replace('\n', '<br />') if is_user else markdown2.markdown(raw_content, extras=["fenced-code-blocks", "break-on-newline", "smarty-pants", "tables", "task_list", "cuddled-lists", "nofollow", "strike", "target-blank-links"])
            chat_html += f"<div class='message-wrapper {wrapper_class}'><div class='sender-name'>{sender_name}</div><div class='message-bubble {bubble_class}'>{processed_content}</div></div>"
        return chat_html

    def atualizar_componentes_ui_com_estado_jogo(estado_jogo_dict, nome_usuario_str="Aventureiro"):
        nomes_habitos = estado_jogo_dict.get("nomes_habitos_para_checkboxgroup", [])
        concluidos_nomes = estado_jogo_dict.get("habitos_concluidos_para_checkboxgroup", [])
        detalhes_md_str = ""
        habitos_detalhados_lista = estado_jogo_dict.get("habitos_detalhados", [])
        if habitos_detalhados_lista:
            detalhes_md_str = "<ul id='detalhes_habitos_lista_ui'>"
            for habito in habitos_detalhados_lista:
                emoji_status = "✅" if habito.get('concluido') else "⏳"
                nome_html = f"<strong>{html.escape(habito.get('nome', 'Missão Desconhecida'))}</strong>"
                meta_html = html.escape(habito.get('meta', 'N/A')); freq_html = html.escape(habito.get('frequencia', 'N/A'))
                xp_val = habito.get('xp', 0); gold_val = habito.get('gold', 0)
                conexao_html = html.escape(habito.get('conexao_busca', 'Sem detalhes de conexão.'))
                detalhes_md_str += (f"<li>{emoji_status} {nome_html}<br><small class='mission-details'>🎯 Meta: {meta_html} | 🗓️ Freq.: {freq_html}<br>✨ Recompensa: {xp_val} XP, {gold_val} Gold<br>🔗 Grande Busca: {conexao_html}</small></li>")
            detalhes_md_str += "</ul>"
        else:
            detalhes_md_str = "<p style='color: #9E9E9E; text-align:center;'>Nenhuma missão no seu pergaminho ainda!</p>"
        nivel_atual = estado_jogo_dict.get('nivel', 1)
        user_info_header = f"Aventureiro: **{html.escape(nome_usuario_str)}** | Nível: **{nivel_atual}**"
        return (gr.update(choices=nomes_habitos, value=concluidos_nomes), detalhes_md_str,
                f"**Nível:** {nivel_atual}", f"**XP:** {estado_jogo_dict.get('xp', 0)}",
                f"**Gold:** {estado_jogo_dict.get('gold', 0)}", user_info_header, estado_jogo_dict)

    def ui_processar_mensagem_e_atualizar_chat(mensagem_usuario_str_input, historico_chat_lista_estado, estado_jogo_dict_estado, nome_usuario_logado_str_estado):
        global conversa_com_ia_para_habitos, xp_total, gold_total, nivel_usuario, plano_de_habitos_gerado, id_habito_counter
        conversa_com_ia_para_habitos = list(historico_chat_lista_estado)
        xp_total = estado_jogo_dict_estado.get("xp", 0); gold_total = estado_jogo_dict_estado.get("gold", 0)
        nivel_usuario = estado_jogo_dict_estado.get("nivel", 1); plano_de_habitos_gerado = list(estado_jogo_dict_estado.get("habitos_detalhados", []))
        id_habito_counter = max([h.get('id',0) for h in plano_de_habitos_gerado] + [0])

        if not mensagem_usuario_str_input or not mensagem_usuario_str_input.strip():
            chat_html_atual = formatar_conversa_para_html(conversa_com_ia_para_habitos)
            (upd_check, upd_det, upd_niv, upd_xp, upd_gold, upd_header, upd_estado_jogo) = atualizar_componentes_ui_com_estado_jogo(estado_jogo_dict_estado, nome_usuario_logado_str_estado)
            return chat_html_atual, conversa_com_ia_para_habitos, upd_check, upd_det, upd_niv, upd_xp, upd_gold, upd_header, upd_estado_jogo

        # Usa INSTRUCAO_SISTEMA_ANALISE_E_PLANOS para todas as interações após a inicial (que é tratada no login)
        resposta_ia_texto, novo_historico_lista_backend = chamar_ia_para_habitos(conversa_com_ia_para_habitos, mensagem_usuario_str_input, INSTRUCAO_SISTEMA_ANALISE_E_PLANOS)
        conversa_com_ia_para_habitos = novo_historico_lista_backend

        chat_html_formatado = formatar_conversa_para_html(conversa_com_ia_para_habitos)
        if not (isinstance(resposta_ia_texto, str) and resposta_ia_texto.startswith("ErroAPI_")):
            if "### 📜 Seu Novo Pergaminho de Missões 📜 ###" in resposta_ia_texto or \
               "### PERGAMINHO DAS MISSÕES INICIAIS ###" in resposta_ia_texto:
                tentar_parsear_plano_de_habitos_da_ia(resposta_ia_texto)

        estado_jogo_final_dict_backend = get_estado_jogo_atualizado()
        (upd_check, upd_det, upd_niv, upd_xp, upd_gold, upd_header, upd_estado_jogo_retorno) = atualizar_componentes_ui_com_estado_jogo(estado_jogo_final_dict_backend, nome_usuario_logado_str_estado)
        return chat_html_formatado, conversa_com_ia_para_habitos, upd_check, upd_det, upd_niv, upd_xp, upd_gold, upd_header, upd_estado_jogo_retorno

    def ui_marcar_habitos_no_checkbox(lista_nomes_habitos_selecionados_ui, estado_jogo_dict_estado, nome_usuario_logado_str_estado):
        global xp_total, gold_total, nivel_usuario, plano_de_habitos_gerado, id_habito_counter
        xp_total = estado_jogo_dict_estado.get("xp", 0); gold_total = estado_jogo_dict_estado.get("gold", 0)
        nivel_usuario = estado_jogo_dict_estado.get("nivel", 1); plano_de_habitos_gerado = list(estado_jogo_dict_estado.get("habitos_detalhados", []))
        id_habito_counter = max([h.get('id',0) for h in plano_de_habitos_gerado] + [0])
        estado_jogo_apos_marcar_backend = marcar_habitos_concluidos_ui(lista_nomes_habitos_selecionados_ui)
        (upd_check, upd_det, upd_niv, upd_xp, upd_gold, upd_header, upd_estado_jogo_retorno) = atualizar_componentes_ui_com_estado_jogo(estado_jogo_apos_marcar_backend, nome_usuario_logado_str_estado)
        return upd_check, upd_det, upd_niv, upd_xp, upd_gold, upd_header, upd_estado_jogo_retorno

    def ui_funcao_inicializacao_login_sucesso(nome_usuario_str_login):
        global conversa_com_ia_para_habitos
        estado_jogo_resetado_dict_backend = resetar_jogo_e_conversa()
        conversa_com_ia_para_habitos = []
        mensagem_inicial_simulada = f"Saudações do Aventureiro {nome_usuario_str_login}!"
        resposta_ia_boas_vindas, historico_inicial_backend = chamar_ia_para_habitos(conversa_com_ia_para_habitos, mensagem_inicial_simulada, INSTRUCAO_SISTEMA_BOAS_VINDAS)
        conversa_com_ia_para_habitos = historico_inicial_backend
        chat_html_inicial_formatado = formatar_conversa_para_html(conversa_com_ia_para_habitos)
        (upd_check, upd_det, upd_niv, upd_xp, upd_gold, upd_header, upd_estado_jogo_retorno) = atualizar_componentes_ui_com_estado_jogo(estado_jogo_resetado_dict_backend, nome_usuario_str_login)
        return (chat_html_inicial_formatado, conversa_com_ia_para_habitos, upd_check, upd_det, upd_niv, upd_xp, upd_gold, upd_header, upd_estado_jogo_retorno)

    def ui_tentar_login(usuario_str_input, senha_str_input):
        usuario_tratado = usuario_str_input.strip().lower()
        senha_tratada = senha_str_input.strip()
        if usuario_tratado == USUARIO_VALIDO and senha_tratada == SENHA_VALIDA:
            outputs_iniciais = ui_funcao_inicializacao_login_sucesso(usuario_tratado)
            # Retorna uma lista/tupla na ordem dos outputs definidos no .click()
            return (
                gr.update(visible=False), # tela_login_ui_wrapper
                gr.update(visible=True),  # tela_principal_habituai_ui
                gr.update(value="Login bem-sucedido!", visible=False), # login_status_msg_ui
                True, # usuario_logado_estado_ui
                usuario_tratado, # nome_usuario_logado_estado_ui
                outputs_iniciais[0], # output_conversa_html_ui
                outputs_iniciais[1], # estado_conversa_hist_ui
                outputs_iniciais[2], # checkbox_habitos_ui_display (gr.update)
                outputs_iniciais[3], # detalhes_habitos_display_markdown_ui
                outputs_iniciais[4], # nivel_display_ui
                outputs_iniciais[5], # xp_display_ui
                outputs_iniciais[6], # gold_display_ui
                outputs_iniciais[8], # estado_jogo_completo_ui (era outputs_iniciais[8])
                outputs_iniciais[7]  # user_info_display_ui (era outputs_iniciais[7])
            )
        else:
            return (
                gr.update(visible=True), gr.update(visible=False),
                gr.update(value="Nome do Aventureiro ou Senha Secreta inválidos!", visible=True),
                False, "",
                "<p style='color:#A0A0A0; text-align:center; margin-top: 20px;'><em>Aguardando login...</em></p>", [],
                gr.update(choices=[], value=[]), "Nenhuma missão ativa.",
                "**Nível:** 1", "**XP:** 0", "**Gold:** 0",
                {"xp": 0, "gold": 0, "nivel": 1, "habitos_detalhados": [], "nomes_habitos_para_checkboxgroup": [], "habitos_concluidos_para_checkboxgroup": []},
                ""
            )

    btn_entrar_ui.click(
        fn=ui_tentar_login,
        inputs=[login_usuario_input_ui, login_senha_input_ui],
        outputs=[
            tela_login_ui_wrapper, tela_principal_habituai_ui, login_status_msg_ui,
            usuario_logado_estado_ui, nome_usuario_logado_estado_ui,
            output_conversa_html_ui, estado_conversa_hist_ui,
            checkbox_habitos_ui_display, detalhes_habitos_display_markdown_ui,
            nivel_display_ui, xp_display_ui, gold_display_ui,
            estado_jogo_completo_ui, # Este deve ser o penúltimo
            user_info_display_ui     # Este deve ser o último
        ]
    )
    btn_enviar_mensagem_ui.click(
        fn=ui_processar_mensagem_e_atualizar_chat,
        inputs=[input_usuario_chat_ui, estado_conversa_hist_ui, estado_jogo_completo_ui, nome_usuario_logado_estado_ui],
        outputs=[output_conversa_html_ui, estado_conversa_hist_ui, checkbox_habitos_ui_display, detalhes_habitos_display_markdown_ui, nivel_display_ui, xp_display_ui, gold_display_ui, user_info_display_ui, estado_jogo_completo_ui]
    ).then(lambda: gr.update(value=""), outputs=input_usuario_chat_ui)
    checkbox_habitos_ui_display.change(
        fn=ui_marcar_habitos_no_checkbox,
        inputs=[checkbox_habitos_ui_display, estado_jogo_completo_ui, nome_usuario_logado_estado_ui],
        outputs=[checkbox_habitos_ui_display, detalhes_habitos_display_markdown_ui, nivel_display_ui, xp_display_ui, gold_display_ui, user_info_display_ui, estado_jogo_completo_ui]
    )

print("Iniciando a interface Gradio do HabitMaster IGNITE com Login (Versão Limpa)...")
demo_habituai_com_login.launch(debug=True, share=True)

Iniciando a interface Gradio do HabitMaster IGNITE com Login (Versão Limpa)...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://67644d4836b5d0f5dd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


DEBUG Backend: Jogo, conversa e objetivo principal resetados.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://67644d4836b5d0f5dd.gradio.live
